In [42]:
# 랜덤포레스트라이브러리
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#geopandas
import geopandas as gpd
import pandas as pd

# 공간연산
from shapely.geometry import Point


In [43]:
# 격지 SHP 파일 읽기
shp_file_grid = "./data/grid3_shp/og_gd03_as.shp"
gdf = gpd.read_file(shp_file_grid)

# 데이터셋

# 데이터 확인
print(gdf.head())  # 데이터 미리 보기
print(gdf.crs)     # 좌표계 정보

         OG_ID                                           geometry
0  GR3_B3A11_A  POLYGON ((120 44, 120.05 44, 120.05 43.95, 120...
1  GR3_B3A11_B  POLYGON ((120.05 44, 120.1 44, 120.1 43.95, 12...
2  GR3_B3A11_C  POLYGON ((120.1 44, 120.15 44, 120.15 43.95, 1...
3  GR3_B3A11_D  POLYGON ((120.15 44, 120.2 44, 120.2 43.95, 12...
4  GR3_B3A11_E  POLYGON ((120.2 44, 120.25 44, 120.25 43.95, 1...
EPSG:4326


In [44]:
gdf

,OG_ID,geometry
0,GR3_B3A11_A,"POLYGON ((120 44, 120.05 44, 120.05 43.95, 120..."
1,GR3_B3A11_B,"POLYGON ((120.05 44, 120.1 44, 120.1 43.95, 12..."
2,GR3_B3A11_C,"POLYGON ((120.1 44, 120.15 44, 120.15 43.95, 1..."
3,GR3_B3A11_D,"POLYGON ((120.15 44, 120.2 44, 120.2 43.95, 12..."
4,GR3_B3A11_E,"POLYGON ((120.2 44, 120.25 44, 120.25 43.95, 1..."
...,...,...
102395,GR3_K2P44_U,"POLYGON ((135.75 28.05, 135.8 28.05, 135.8 28,..."
102396,GR3_K2P44_V,"POLYGON ((135.8 28.05, 135.85 28.05, 135.85 28..."
102397,GR3_K2P44_W,"POLYGON ((135.85 28.05, 135.9 28.05, 135.9 28,..."
102398,GR3_K2P44_X,"POLYGON ((135.9 28.05, 135.95 28.05, 135.95 28..."


In [45]:
def process_de(dataframe):
    """
    데이터프레임에서 특정 열의 null 값을 제거하고,
    필요한 컬럼만 남긴 후 열 이름을 변경하는 함수.
    """
    # Null 값 제거: 컬럼 이름이 '구조위도(TM구조위도)' 또는 '구조위도'일 경우
    if '구조위도(TM구조위도)' in dataframe.columns:
        dataframe = dataframe.dropna(subset=['구조위도(TM구조위도)'])
    elif '구조위도' in dataframe.columns:
        dataframe = dataframe.dropna(subset=['구조위도'])
    
    # 특정 컬럼만 선택
    if '구조위도(TM구조위도)' in dataframe.columns and '구조경도(TM구조경도)' in dataframe.columns:
        dataframe = dataframe[['구조일시', '구조위도(TM구조위도)', '구조경도(TM구조경도)']]
    elif '구조위도' in dataframe.columns and '구조경도' in dataframe.columns:
        dataframe = dataframe[['구조일시', '구조위도', '구조경도']]

    # 열 이름 변경: '구조위도(TM구조위도)'와 '구조경도(TM구조위도)'가 존재하는 경우
    if '구조위도(TM구조위도)' in dataframe.columns or '구조경도(TM구조경도)' in dataframe.columns:
        dataframe.rename(columns={
            '구조위도(TM구조위도)': '구조위도',
            '구조경도(TM구조경도)': '구조경도'
        }, inplace=True)
    
    return dataframe


In [46]:
# CSV 파일 불러오기+전처리
de_df1 = process_de(pd.read_csv("./data/충청남도_계룡시 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df2 = process_de(pd.read_csv("./data/충청남도_공주시 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df3 = process_de(pd.read_csv("./data/충청남도_금산군 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df4 = process_de(pd.read_csv("./data/충청남도_논산시 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df5 = process_de(pd.read_csv("./data/충청남도_당진시 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df6 = process_de(pd.read_csv("./data/충청남도_보령시 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df7 = process_de(pd.read_csv("./data/충청남도_부여군 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df8 = process_de(pd.read_csv("./data/충청남도_서산시 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df9 = process_de(pd.read_csv("./data/충청남도_서천군 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df10 = process_de(pd.read_csv("./data/충청남도_아산시 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df11 = process_de(pd.read_csv("./data/충청남도_예산군 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df12 = process_de(pd.read_csv("./data/충청남도_천안시 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df13 = process_de(pd.read_csv("./data/충청남도_청양군 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df14 = process_de(pd.read_csv("./data/충청남도_태안군 야생동물 구조데이터_20230630.csv", encoding='cp949'))
de_df15 = process_de(pd.read_csv("./data/충청남도_홍성군 야생동물 구조데이터_20230713.csv", encoding='cp949'))

In [47]:
# 데이터프레임 리스트
dataframes = [de_df1, de_df2, de_df3, de_df4, de_df5, 
              de_df6, de_df7, de_df8, de_df9, de_df10, 
              de_df11, de_df12, de_df13, de_df14, de_df15]

# 빈 데이터프레임 생성 (각 연도를 저장할 데이터프레임)
de_df_2019 = pd.DataFrame()
de_df_2020 = pd.DataFrame()
de_df_2021 = pd.DataFrame()

# 데이터프레임 분리 및 합치기
for df in dataframes:
    if '구조일시' in df.columns:
        # 2019년도 데이터
        df_2019 = df[df['구조일시'].str.startswith('2019')]
        de_df_2019 = pd.concat([de_df_2019, df_2019], ignore_index=True)

        # 2020년도 데이터
        df_2020 = df[df['구조일시'].str.startswith('2020')]
        de_df_2020 = pd.concat([de_df_2020, df_2020], ignore_index=True)

        # 2021년도 데이터
        df_2021 = df[df['구조일시'].str.startswith('2021')]
        de_df_2021 = pd.concat([de_df_2021, df_2021], ignore_index=True)


In [48]:
de_df_2019

,구조일시,구조위도,구조경도
0,2019-07-29,36 17 10.0,127 14 22.9
1,2019-05-03,36 17 16.2,127 13 47.2
2,2019-05-03,36 17 16.2,127 13 47.2
3,2019-05-03,36 17 16.2,127 13 47.2
4,2019-06-28,36 19 02.1,127 14 25.1
...,...,...,...
1235,2019-10-16,36 33 42.6,126 41 42.6
1236,2019-10-16,36 33 59.7,126 38 57.9
1237,2019-10-25,36 33 42.5,126 38 54.2
1238,2019-10-28,36 39 22.3,126 39 58.9


In [49]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# DMS 형식을 Decimal 형식으로 변환하는 함수
def dms_to_decimal(dms):
    try:
        parts = dms.split()
        if len(parts) == 3:  # DMS 형식
            degrees, minutes, seconds = map(float, parts)
            return degrees + (minutes / 60) + (seconds / 3600)
        elif len(parts) == 1:  # 이미 Decimal 형식
            return float(dms)
        else:
            raise ValueError("Invalid DMS format")
    except Exception as e:
        print(f"Invalid value: {dms} - {e}")
        return None

# 1. de_df_2019의 구조위도와 구조경도를 Decimal 형식으로 변환
de_df_2019['구조위도'] = de_df_2019['구조위도'].apply(dms_to_decimal)
de_df_2019['구조경도'] = de_df_2019['구조경도'].apply(dms_to_decimal)

de_df_2020['구조위도'] = de_df_2020['구조위도'].apply(dms_to_decimal)
de_df_2020['구조경도'] = de_df_2020['구조경도'].apply(dms_to_decimal)

de_df_2021['구조위도'] = de_df_2021['구조위도'].apply(dms_to_decimal)
de_df_2021['구조경도'] = de_df_2021['구조경도'].apply(dms_to_decimal)

# 2. de_df_2019를 GeoDataFrame으로 변환 (좌표 포인트 생성)
de_gdf_2019 = gpd.GeoDataFrame(
    de_df_2019,
    geometry=gpd.points_from_xy(de_df_2019['구조경도'], de_df_2019['구조위도']),
    crs="EPSG:4326"
)
de_gdf_2020 = gpd.GeoDataFrame(
    de_df_2020,
    geometry=gpd.points_from_xy(de_df_2020['구조경도'], de_df_2020['구조위도']),
    crs="EPSG:4326"
)
de_gdf_2021 = gpd.GeoDataFrame(
    de_df_2021,
    geometry=gpd.points_from_xy(de_df_2021['구조경도'], de_df_2021['구조위도']),
    crs="EPSG:4326"
)

# 3. gdf와 de_gdf_2019의 공간 조인
# gdf에 포함된 de_gdf_2019 포인트를 count
gdf = gdf.set_crs("EPSG:4326")  # 좌표계 설정
joined = gpd.sjoin(de_gdf_2019, gdf, how='inner', predicate='within')
joined20 = gpd.sjoin(de_gdf_2020, gdf, how='inner', predicate='within')
joined21 = gpd.sjoin(de_gdf_2021, gdf, how='inner', predicate='within')

# 4. id와 count 계산
target_2019 = joined.groupby('OG_ID').size().reset_index(name='count')
target_2020 = joined20.groupby('OG_ID').size().reset_index(name='count')
test_2021 = joined21.groupby('OG_ID').size().reset_index(name='count')

# id 컬럼 이름 변경
target_2019.rename(columns={'OG_ID': 'id'}, inplace=True)
target_2020.rename(columns={'OG_ID': 'id'}, inplace=True)
test_2021.rename(columns={'OG_ID': 'id'}, inplace=True)

# 결과 확인
print(target_2019)

              id  count
0    GR3_F2K34_W      3
1    GR3_F2K43_P      1
2    GR3_F2K43_U      1
3    GR3_F2K43_V      1
4    GR3_F2O11_O      4
..           ...    ...
227  GR3_F2P43_A      1
228  GR3_F2P43_B      1
229  GR3_F2P43_F      1
230  GR3_F2P43_L      1
231  GR3_F4C14_P      1

[232 rows x 2 columns]


In [50]:
# target_2019를 CSV 파일로 저장
output_file = "target_2019_종속변수.csv"
target_2019.to_csv(output_file, index=False, encoding='utf-8-sig')  # UTF-8-sig로 인코딩

output_file = "target_2020_종속변수.csv"
target_2020.to_csv(output_file, index=False, encoding='utf-8-sig')  # UTF-8-sig로 인코딩

output_file = "test_2021_종속변수.csv"
test_2021.to_csv(output_file, index=False, encoding='utf-8-sig')  # UTF-8-sig로 인코딩

print(f"'{output_file}' 파일로 저장되었습니다.")

'test_2021_종속변수.csv' 파일로 저장되었습니다.
